In [ ]:
service_credential = dbutils.secrets.get(scope="databricksscopename",key="databricksservicekey")

spark.conf.set("fs.azure.account.auth.type.adlsx5u224gv3xgd2.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.adlsx5u224gv3xgd2.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.adlsx5u224gv3xgd2.dfs.core.windows.net", "5c962712-e111-48b0-b9af-41f03175e216")
spark.conf.set("fs.azure.account.oauth2.client.secret.adlsx5u224gv3xgd2.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.adlsx5u224gv3xgd2.dfs.core.windows.net", "https://login.microsoftonline.com/72f988bf-86f1-41af-91ab-2d7cd011db47/oauth2/token")

In [ ]:
# Source with default settings
%pip install azure-eventhub
from azure.eventhub import EventHubProducerClient, EventHubConsumerClient, EventData

import pyspark.pandas as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *



from pyspark import SparkContext
sc =SparkContext.getOrCreate()

connectionString = "Endpoint=sb://streamdata-84d2c3-ns.servicebus.windows.net/;SharedAccessKeyName=rule;SharedAccessKey=MjLN4fNXCODO9T/eNCXJObufnUu8yIXoA+AEhNzpTXQ=;EntityPath=streamdata-84d2c3-ns"
ehConf = {}
ehConf['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString)

df = spark \
  .readStream \
  .format("eventhubs") \
  .options(**ehConf) \
  .load()

sales_orders_df = df.select("body").withColumn("body", col("body").cast("string"))

sales_orders_df.writeStream.format("delta")\
   .outputMode("append")\
    .option('path','abfss://data@adlsx5u224gv3xgd2.dfs.core.windows.net/retail_org_dt/Bronze/sales_order_stream')\
   .option("checkpointLocation", "/tmp/delta/events/_checkpoints/")\
   .toTable("retail_org_dt.bronze_sales_order_stream")



In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

sales_orders_schema = StructType(
    [
        StructField(
            "clicked_items", ArrayType(ArrayType(StringType(), True), True), True
        ),
        StructField("customer_id", LongType(), True),
        StructField("customer_name", StringType(), True),
        StructField("number_of_line_items", LongType(), True),
        StructField("order_datetime", StringType(), True),
        StructField("order_number", LongType(), True),
        StructField(
            "ordered_products",
            ArrayType(
                StructType(
                    [
                        StructField("id", StringType(), True),
                        StructField("qty", IntegerType(), True),
                        StructField("curr", StringType(), True),
                        StructField("name", StringType(), True),
                        StructField("unit", StringType(), True),
                        StructField("price", IntegerType(), True),
                        StructField(
                            "promotion_info",
                            StructType(
                                [
                                    StructField("promo_id", IntegerType(), True),
                                    StructField("promo_qty", IntegerType(), True),
                                    StructField("promo_disc", DecimalType(3, 2), True),
                                    StructField("promo_item", StringType(), True),
                                ]
                            ),
                            True,
                        ),
                    ]
                ),
                True,
            ),
            True,
        ),
        StructField(
            "promo_info",
            ArrayType(
                StructType(
                    [
                        StructField("promo_id", IntegerType(), True),
                        StructField("promo_qty", IntegerType(), True),
                        StructField("promo_disc", DecimalType(3, 2), True),
                        StructField("promo_item", StringType(), True),
                    ]
                ),
                True,
            ),
            True,
        ),
    ]
  )

df1 = (spark.readStream.format("delta").table("retail_org_dt.bronze_sales_order_stream")
       .select("*")
       .withColumn("body", regexp_replace("body", '"\\[', "["))
       .withColumn("body", regexp_replace("body", '\\]"', "]"))
       .withColumn("body", regexp_replace("body","\\\\", ""))
       .withColumn("body", expr("substring(body, 2, length(body))"))
       .withColumn("body", expr("substring(body, 1, length(body)-1)"))
       .select(from_json(col("body"), sales_orders_schema).alias("row"))
       .select("row.*")
       .withColumn("ordered_products", explode("ordered_products"))
       .withColumn("order_datetime", from_unixtime("order_datetime"))
       .withColumn("product_id", col("ordered_products").id)
       .withColumn("unit_price", col("ordered_products").price)
       .withColumn("quantity", col("ordered_products").qty)
       
      )
df1.writeStream.format("delta")\
   .outputMode("append")\
   .option('path','abfss://data@adlsx5u224gv3xgd2.dfs.core.windows.net/retail_org_dt/Silver/sales_orders')\
   .option("checkpointLocation", "/tmp/delta/eventschkforsilver/_checkpoints/")\
   .toTable("retail_org_dt.silver_sales_orders")

In [ ]:
s = spark.readStream.format("delta").table("retail_org_dt.silver_sales_orders").alias("s")
p = spark.read.format("delta").table("retail_org_dt.dim_products").alias("p")
c = spark.read.format("delta").table("retail_org_dt.dim_customers").alias("c")

df2 = s.join(p, s.product_id == p.product_id, "inner").join(c, s.customer_id == c.customer_id, "inner").select("s.order_number","c.customer_key","p.product_key",col("s.order_datetime").cast("date").alias("order_date"),"s.unit_price","s.quantity",expr("s.unit_price * s.quantity").alias("total_price"),)
df2.writeStream.format("delta")\
   .outputMode("append")\
   .option('path','abfss://data@adlsx5u224gv3xgd2.dfs.core.windows.net/retail_org_dt/Gold/fact_customer_sales_stream')\
   .option("checkpointLocation", "/tmp/delta/eventscheckforgold/_checkpoints/")\
   .toTable("retail_org_dt.fact_customer_sales_stream")